In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

pd.options.display.max_columns=100



In [2]:
df = pd.read_csv('holdout_data.csv')

In [3]:
df_columns = pd.read_csv('training_data.csv' , index_col=0)
df_columns.drop(['Y'], axis=1, inplace=True)

In [4]:
df_columns.reset_index(inplace=True)

In [5]:
print(len(df.columns))
print(len(df_columns.columns))

24
24


In [6]:
df.columns=list(df_columns.iloc[18381])
df=df[df.ID!="ID"]
df.reset_index(drop=True,inplace=True)
df=df.rename(columns = {'default payment next month':'Y'})

df=df.astype(int)

/Users/carlosruiz/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [7]:
df.MARRIAGE = df.MARRIAGE.apply(lambda x: 3 if x==0 else x)
df.MARRIAGE = df.MARRIAGE.apply(lambda x: 2 if x==3 else x)

In [8]:
conditions = [
    df.EDUCATION==0,
    df.EDUCATION==6
]

choices = [
    5,
    5
]

df.EDUCATION = np.select(conditions, choices, default=df.EDUCATION)

In [9]:
conditions = [
    (df.AGE>=20)&(df.AGE<30),
    (df.AGE>=30)&(df.AGE<40),
    (df.AGE>=40)&(df.AGE<50),
    (df.AGE>=50)&(df.AGE<60),
    (df.AGE>=60)&(df.AGE<70),
    (df.AGE>=70)&(df.AGE<80)
]

choices = [
    '20s',
    '30s',
    '40s',
    '50s',
    '60s',
    '70s'
]

df.AGE = np.select(conditions, choices)

In [10]:
df['MON_OV_BAL']=0

In [11]:
conditions=[
    df.BILL_AMT1>=df.LIMIT_BAL,
    df.BILL_AMT2>=df.LIMIT_BAL,
    df.BILL_AMT3>=df.LIMIT_BAL,
    df.BILL_AMT4>=df.LIMIT_BAL,
    df.BILL_AMT5>=df.LIMIT_BAL,
    df.BILL_AMT6>=df.LIMIT_BAL,
]


hold_mon_ov=pd.DataFrame(np.where(conditions,1,0) )
tot_mon_ov=[]
for i in hold_mon_ov.columns:
    tot_mon_ov.append(hold_mon_ov[i].sum())


df['MON_OV_BAL']=tot_mon_ov

In [12]:
df['SUM_PAY_STAT']=df[['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']].sum(axis=1)

In [13]:
conditions=[
    df.BILL_AMT2>=df.BILL_AMT1,
    df.BILL_AMT3>=df.BILL_AMT2,
    df.BILL_AMT4>=df.BILL_AMT3,
    df.BILL_AMT5>=df.BILL_AMT4,
    df.BILL_AMT6>=df.BILL_AMT5,
]

hold_mon_bill=pd.DataFrame(np.where(conditions,1,0) )
tot_mon_bill=[]
for i in hold_mon_bill.columns:
    tot_mon_bill.append(hold_mon_bill[i].sum())

df['MON_BIL_INC']=tot_mon_bill

df=pd.get_dummies(df,columns=['MON_BIL_INC'],drop_first=True)

In [14]:
total=[]
for i in range(len(df)):
    total.append(list(df[['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']].loc[i].mode()))
total=[x[0]for x in total]
df['PAY_STAT_MODE']=total

In [15]:
df['BILL_AV']=df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].mean(axis=1)

In [16]:
df['PAY_AV']=df[['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']].mean(axis=1)

In [17]:
df['AVG_UTILIZATION'] = df.BILL_AV/df.LIMIT_BAL

In [18]:
df['AVG_PERCENT_PAID'] = df.PAY_AV/df.BILL_AV

In [19]:
df.AVG_PERCENT_PAID.fillna(value=0, inplace=True)

In [20]:
df.AVG_PERCENT_PAID.replace(np.inf, value=0, inplace=True)

In [21]:
df=pd.get_dummies(df,columns=['AGE'],drop_first=True)

In [22]:
df['PERCENT_USAGE']=(df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].sum(axis=1)*600)/(df['LIMIT_BAL']*6)

In [23]:
df['BEEN_LATE']=0
conditions=[
(df.PAY_0>=1)|(df.PAY_2>=1)|(df.PAY_3>=1)|(df.PAY_4>=1)|(df.PAY_5>=1)|(df.PAY_6>=1),

]
choises = [
    1,
]
df['BEEN_LATE']=np.select(conditions,choises,default=0)   
      

In [24]:
df['TOTAL_USAGE']=(df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].sum(axis=1))

In [25]:
df['TOTAL_PAY']=(df[['PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']].sum(axis=1))

In [26]:

conditions=[
    df.BILL_AMT1>df.LIMIT_BAL,
    df.BILL_AMT2>df.LIMIT_BAL,
    df.BILL_AMT3>df.LIMIT_BAL,
    df.BILL_AMT4>df.LIMIT_BAL,
    df.BILL_AMT5>df.LIMIT_BAL,
    df.BILL_AMT6>df.LIMIT_BAL,

]


hold=np.where(conditions,1,0) 


df['OV_BILL1']=hold[0]
df['OV_BILL2']=hold[1]
df['OV_BILL3']=hold[2]
df['OV_BILL4']=hold[3]
df['OV_BILL5']=hold[4]
df['OV_BILL6']=hold[5]

In [27]:
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,MON_OV_BAL,SUM_PAY_STAT,MON_BIL_INC_1,MON_BIL_INC_2,MON_BIL_INC_3,MON_BIL_INC_4,MON_BIL_INC_5,PAY_STAT_MODE,BILL_AV,PAY_AV,AVG_UTILIZATION,AVG_PERCENT_PAID,AGE_30s,AGE_40s,AGE_50s,AGE_60s,AGE_70s,PERCENT_USAGE,BEEN_LATE,TOTAL_USAGE,TOTAL_PAY,OV_BILL1,OV_BILL2,OV_BILL3,OV_BILL4,OV_BILL5,OV_BILL6
0,5501,180000,2,2,1,0,0,0,0,0,0,161186,167080,170788,174764,162667,166953,10000,8000,7000,6000,7000,10000,0,0,0,0,0,1,0,0,167239.666667,8000.000000,0.929109,0.047836,0,1,0,0,0,557.465556,0,1003438,48000,0,0,0,0,0,0
1,28857,130000,2,2,1,-2,-2,-2,-2,-2,-2,0,1240,1487,1279,749,440,1240,1487,1279,749,440,849,0,-12,0,1,0,0,0,-2,865.833333,1007.333333,0.006660,1.163426,0,1,0,0,0,3.996154,0,5195,6044,0,0,0,0,0,0
2,11272,60000,2,1,1,-1,3,2,0,0,-1,495,330,495,330,165,340,0,330,0,0,340,0,0,3,0,1,0,0,0,-1,359.166667,111.666667,0.005986,0.310905,0,1,0,0,0,3.591667,1,2155,670,0,0,0,0,0,0
3,8206,240000,1,1,1,0,0,0,0,0,0,72339,91045,91027,51508,51127,0,20000,2213,1030,1023,6790,10893,0,0,1,0,0,0,0,0,59507.666667,6991.500000,0.247949,0.117489,0,1,0,0,0,148.769167,0,357046,41949,0,0,0,0,0,0
4,6362,100000,2,2,1,2,0,0,0,0,2,73073,74739,70844,63924,57326,59654,3500,3003,1910,2400,3300,0,0,4,0,1,0,0,0,0,66593.333333,2352.166667,0.665933,0.035321,0,0,0,0,0,399.560000,1,399560,14113,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,14600,90000,2,2,1,-2,-2,-2,-2,-2,-2,1905,1924,11855,665,0,665,1924,11855,10655,0,665,0,0,-12,0,0,1,0,0,-2,2835.666667,4183.166667,0.031507,1.475197,1,0,0,0,0,18.904444,0,17014,25099,0,0,0,0,0,0
7497,12687,180000,2,2,2,0,0,0,0,0,0,103203,108032,109741,112907,115924,118832,6500,5000,5000,5000,5000,5000,0,0,0,0,0,0,1,0,111439.833333,5250.000000,0.619110,0.047111,0,0,0,0,0,371.466111,0,668639,31500,0,0,0,0,0,0
7498,7374,360000,1,2,1,1,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,-9,0,0,0,0,1,-2,0.000000,0.000000,0.000000,0.000000,1,0,0,0,0,0.000000,1,0,0,0,0,0,0,0,0
7499,27661,50000,2,2,2,-1,0,0,2,0,0,9101,10446,12595,11449,9914,9875,1502,2651,500,500,500,500,0,1,0,1,0,0,0,0,10563.333333,1025.500000,0.211267,0.097081,0,0,0,0,0,126.760000,1,63380,6153,0,0,0,0,0,0


In [29]:
import pickle
Model = pickle.load(open("Model.pickle", 'rb'))

In [30]:
final_answers = Model.predict(df)

In [36]:
final_answers= pd.DataFrame(final_answers)
final_answers.columns=['Predictions']

In [37]:
final_answers.

,Predictions
0,0
1,0
2,1
3,0
4,1
...,...
7496,0
7497,0
7498,1
7499,1


In [38]:
final_answers.to_csv('predictions_carlos_ruiz.csv')